In [1]:
# import torch
# from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig



# base_model_id = "meta-llama/Llama-2-13b-hf"
# # bnb_config = BitsAndBytesConfig(
# #     load_in_4bit=True,
# #     bnb_4bit_use_double_quant=True,
# #     bnb_4bit_quant_type="nf4",
# #     bnb_4bit_compute_dtype=torch.bfloat16
# # )


# #model = AutoModelForCausalLM.from_pretrained(base_model_id, quantization_config=bnb_config)



# quantization_config = BitsAndBytesConfig(llm_int8_enable_fp32_cpu_offload= True )


# base_model = AutoModelForCausalLM.from_pretrained(
#     base_model_id,  
#     quantization_config=bnb_config,  # Same quantization config as before
#     device_map="auto",
#     trust_remote_code=True,
#     use_auth_token=True
# )

# tokenizer = AutoTokenizer.from_pretrained(base_model_id, add_bos_token=True, trust_remote_code=True)

# # import torch
# # from transformers import AutoTokenizer, AutoModelForCausalLM

# # base_model_id = "meta-llama/Llama-2-13b-hf"
# # # 加载基础模型和tokenizer
# # tokenizer = AutoTokenizer.from_pretrained(base_model_id)
# # base_model = AutoModelForCausalLM.from_pretrained(base_model_id)


In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# Define the base model ID
base_model_id = "meta-llama/Llama-2-13b-hf"

# Create a BitsAndBytesConfig object with the corrected settings
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    load_in_8bit_fp32_cpu_offload=True  # Set as suggested in the error
)

# Load the base model with the updated quantization configuration
# Adjust 'device_map' based on your system's GPU configuration
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_id,  
    quantization_config=quantization_config,
    trust_remote_code=True,
    use_auth_token=True
)

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model_id, add_bos_token=True, trust_remote_code=True)


/home/chuning/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/chuning/miniconda3/envs/llama/lib/python3.10/site-packages/transformers/models/auto/auto_factory.py:472: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
Loading checkpoint shards: 100%|██████████| 3/3 [00:10<00:00,  3.62s/it]
/home/chuning/miniconda3/envs/llama/lib/python3.10/site-packages/transformers/utils/hub.py:374: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


In [3]:
eval_prompt = """
As a contract RTL design engineer, you will work as part of a GPU IP design team. This is an open-ended performance-driven position where the contractor will be tasked with supporting RTL design of various sub-blocks of the GPU graphics pipeline for a GPU targeted to the mobile market as well as other markets. Significant architectural, as well as RTL design experience. Key responsibilities include: •Micro-architecture and RTL definition •Partner with the physical design team to resolve implementation level details •Work closely with design verification to test plan and otherwise ensure proper functionality requirements •Deliver quality micro-architectural level documentation •Produce quality RTL on schedule meeting PPA goals. Optimize design for low power.\n\n\nRequirements\nMinimum requirements: •BSEE, Computer Engineer or comparable and 3 + years of experience •Experience driving the RTL design of various digital sub-blocks of GPU /CPU high-performance digital designs •Demonstrated experience of successful Architectural through RTL design experience on high-performance digital designs The preferred candidate will possess the following: •Good written and verbal communication skills •Pixel Pipe or GPU experience preferred •Low Power Design experience preferred"

"""
model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

base_model.eval()
with torch.no_grad():
    print(tokenizer.decode(base_model.generate(**model_input, max_new_tokens=256, pad_token_id=2)[0], skip_special_tokens=True))



As a contract RTL design engineer, you will work as part of a GPU IP design team. This is an open-ended performance-driven position where the contractor will be tasked with supporting RTL design of various sub-blocks of the GPU graphics pipeline for a GPU targeted to the mobile market as well as other markets. Significant architectural, as well as RTL design experience. Key responsibilities include: •Micro-architecture and RTL definition •Partner with the physical design team to resolve implementation level details •Work closely with design verification to test plan and otherwise ensure proper functionality requirements •Deliver quality micro-architectural level documentation •Produce quality RTL on schedule meeting PPA goals. Optimize design for low power.


Requirements
Minimum requirements: •BSEE, Computer Engineer or comparable and 3 + years of experience •Experience driving the RTL design of various digital sub-blocks of GPU /CPU high-performance digital designs •Demonstrated exp

Now load the QLoRA adapter from the appropriate checkpoint directory, i.e. the best performing model checkpoint:

In [4]:
from peft import PeftModel

ft_model = PeftModel.from_pretrained(base_model, "/home/chuning/Downloads/llama/llama2-13b-IPG-finetune/checkpoint-800")

In [5]:



# model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

# ft_model.eval()
# with torch.no_grad():
#     print(tokenizer.decode(ft_model.generate(**model_input, max_new_tokens=300)[0], skip_special_tokens=True))

In [6]:
from datasets import load_dataset

  
eval_dataset = load_dataset('json', data_files='/home/chuning/Downloads/llama/test/new_test_data.json', split='train')


def formatting_func(example):
    text = f"### The job description: {example['text']}\n ### The skills: "
    return text



In [7]:


def run_finetune_model(model_id):

    example = eval_dataset.filter(lambda x: x['id'] == model_id)[0]
    formatted_text = formatting_func(example)
    
    #print(formatted_text)
    model_input = tokenizer(formatted_text, return_tensors="pt").to("cuda")


    ft_model.eval()
    with torch.no_grad():
        output_tokens = ft_model.generate(**model_input, max_new_tokens=150)[0]
        generated_text = tokenizer.decode(output_tokens, skip_special_tokens=True)
    
    print(generated_text)




In [8]:
run_finetune_model("15503")


### The job description: Senior Software Engineer - Payroll (San Francisco / Remote (US))
About Rippling
Rippling is the first way for businesses to manage all of their HR & IT —  payroll, benefits, computers, apps, and more — in one unified workforce platform.
By connecting every workforce system to a single source of truth for employee data, businesses can automate all of the manual work they normally need to do to make employee changes. Take onboarding, for example. With Rippling, you can just click a button and set up a new employees' payroll, health insurance, work computer, and third-party apps — like Slack, Zoom, and Office 365 — all within 90 seconds.
Based in San Francisco, CA, Rippling has raised $450M from the world’s top investors—including Kleiner Perkins, Founders Fund, and Sequoia—and was named one of America's best startup employers by Forbes (#12 out of 500).
About The Role
Rippling has 15+ different products and Payroll is the core offering that is used by almost all 